# COMS W4705 Spring 24
## Homework 4 - Semantic Role Labelling with BERT

The goal of this assignment is to train and evaluate a PropBank-style semantic role labeling (SRL) system. Following (Collobert et al. 2011) and others, we will treat this problem as a sequence-labeling task. For each input token, the system will predict a B-I-O tag, as illustrated in the following example: 

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|||schedule.01|||||||||||||

Note that the same sentence may have multiple annotations for different predicates

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|O|B-V|B-ARG2|I-ARG2|I-ARG2|B-ARGM-TMP|O|
||||||||||remove.01||||||

and not all predicates need to be verbs

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|    
|O|O|O|O|O|O|B-ARG1|B-V|O|O|O|O|O|O|O|
||||||||try.02||||||||

The SRL system will be implemented in [PyTorch](https://pytorch.org/). We will use BERT (in the implementation provided by the [Huggingface transformers](https://huggingface.co/docs/transformers/index) library) to compute contextualized token representations and a custom classification head to predict semantic roles. We will fine-tune the pretrained BERT model on the SRL task. 


### Overview of the Approach

The model we will train is pretty straightforward. Essentially, we will just encode the sentence with BERT, then take the contextualized embedding for each token and feed it into a classifier to predict the corresponding tag. 

Because we are only working on argument identification and labeling (not predicate identification), it is essentially that we tell the model where the predicate is. This can be accomplished in various ways. The approach we will choose here repurposes Bert's *segment embeddings*. 

Recall that BERT is trained on two input sentences, seperated by [SEP], and on a next-sentence-prediction objective (in addition to the masked LM objective). To help BERT comprehend which sentence a given token belongs to, the original BERT uses a segment embedding, using A for the first sentene, and B for the second sentence 2. 
Because we are labeling only a single sentence at a time, we can use the segment embeddings to indicate the predicate position instead: The predicate is labeled as segment B (1) and all other tokens will be labeled as segment A (0). 

<img src="https://github.com/daniel-bauer/4705-f23-hw5/blob/main/bert_srl_model.png?raw=true" width=400px>

## Setup: GCP, Jupyter, PyTorch, GPU

To make sure that PyTorch is available and can use the GPU,run the following cell which should return True. If it doesn't, make sure the GPU drivers and CUDA are installed correctly. 

GPU support is required for this assignment -- you will not be able to fine-tune BERT on a CPU. 

In [6]:
import torch
torch.cuda.is_available() 

True

## Dataset: Ontonotes 5.0 English SRL annotations

We will work with the English part of the [Ontonotes 5.0](https://catalog.ldc.upenn.edu/LDC2013T19) data. This is an extension of PropBank, using the same type of annotation. Ontonotes contains annotations other than predicate/argument structures, but we will use the PropBank style SRL annotations only. *Important*: This data set is provided to you for use in COMS 4705 only! Columbia is a subscriber to LDC and is allowed to use the data for educational purposes. However, you may not use the dataset in projects unrelated to Columbia teaching or research.

If you haven't done so already, you can download the data here: 


In [ ]:
! wget https://storage.googleapis.com/4705-bert-srl-data/ontonotes_srl.zip    

In [ ]:
! unzip ontonotes_srl.zip    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Archive:  ontonotes_srl.zip
replace propbank_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

The data has been pre-processed in the following format. There are three files: 

`propbank_dev.tsv`	`propbank_test.tsv`	`propbank_train.tsv`

Each of these files is in a tab-separated value format. A single predicate/argument structure annotation consists of four rows. For example 

```
ontonotes/bc/cnn/00/cnn_0000.152.1
The     judge   scheduled       to      preside over    his     trial   was     removed from    the     case    today   /.
                schedule.01
B-ARG1  I-ARG1  B-V     B-ARG2  I-ARG2  I-ARG2  I-ARG2  I-ARG2  O       O       O       O       O       O       O
```

* The first row is a unique identifier (1st annotation of the 152nd sentence in the file ontonotes/bc/cnn/00/cnn_0000).
* The second row contains the tokens of the sentence (tab-separated). 
* The third row contains the probank frame name for the predicate (empty field for all other tokens). 
* The fourth row contains the B-I-O tag for each token. 

The file `rolelist.txt` contains a list of propbank BIO labels in the dataset (i.e. possible output tokens). This list has been filtered to contain only roles that appeared more than 1000 times in the training data. 
We will load this list and create mappings from numeric ids to BIO tags and back.

In [7]:
role_to_id = {}
with open("role_list.txt",'r') as f:
    role_list = [x.strip() for x in f.readlines()]
    role_to_id = dict((role, index) for (index, role) in enumerate(role_list))
    role_to_id['[PAD]'] = -100
    
    id_to_role = dict((index, role) for (role, index) in role_to_id.items())

Note that we are also mapping the '[PAD]' token to the value -100. This allows the loss function to ignore these tokens during training.

## Part 1 - Data Preparation

Before you can build the SRL model, you first need to preprocess the data. 


### 1.1 - Tokenization 

One challenge is that the pre-trained BERT model uses subword ("WordPiece") tokenization, but the Ontonotes data does not. Fortunately Huggingface transformers provides a tokenizer. 

In [8]:
from transformers import BertTokenizerFast 
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize("This is an unbelievably boring test sentence.")

['this',
 'is',
 'an',
 'un',
 '##bel',
 '##ie',
 '##va',
 '##bly',
 'boring',
 'test',
 'sentence',
 '.']

**TODO**: 
We need to be able to maintain the correct labels (B-I-O tags) for each of the subwords. 
Complete the following function that takes a list of tokens and a list of B-I-O labels of the same length as parameters, and returns a new token / label pair, as illustrated in the following example. 


```
>>> tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)
(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

```

To approach this problem, iterate through each word/label pair in the sentence. Call the tokenizer on the word. This may result in one or more tokens. Create the correct number of labels to match the number of tokens. Take care to not generate multiple B- tokens. 


This approach is a bit slower than tokenizing the entire sentence, but is necessary to produce proper input tokenization for the pre-trained BERT model, and the matching target labels. 

In [9]:
def tokenize_with_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. 
    """

    tokenized_sentence = []
    labels = []
    
    for word, label in zip(sentence, text_labels):

        count = 0
        for subtoken in tokenizer.tokenize (word):
            tokenized_sentence.append (subtoken)
            if (count == 0):
                labels.append(label)
            else:
                if (len (label) > 2):
                    labels.append ('I' + label [1:])
                else:
                    labels.append ('O')
            count += 1

    return tokenized_sentence, labels

In [10]:
tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)

(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

### 1.2 Loading the Dataset

Next, we are creating a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. This class acts as a contained for the training, development, and testing data in memory. You should already be familiar with Datasets and Dataloaders from homework 3. 

1.2.1 **TODO**: Write the \_\_init\_\_(self, filename) method that reads in the data from a data file (specified by the filename).

For each annotation you start with  the tokens in the sentence, and the BIO tags. Then you need to create the following 

1. call the `tokenize_with_labels` function to tokenize the sentence.
2. Add the (token, label) pair to the self.items list. 

1.2.2 **TODO**: Write the \_\_len\_\_(self) method that returns the total number of items. 

1.2.3 **TODO**: Write the \_\_getitem\_\_(self, k) method that returns a single item in a format BERT will understand. 
* We need to process the sentence by adding "\[CLS\]" as the first token and "\[SEP\]" as the last token. The need to pad the token sequence to 128 tokens using the "\[PAD\]" symbol. This needs to happen both for the inputs (sentence token sequence) and outputs (BIO tag sequence).
* We need to create an *attention mask*, which is a sequence of 128 tokens indicating the actual input symbols (as a 1) and \[PAD\] symbols (as a 0).
* We need to create a *predicate indicator* mask, which is a sequence of 128 tokens with at most one 1, in the position of the "B-V" tag. All other entries should be 0. The model will use this information to understand where the predicate is located. 

* Finally, we need to convert the token and tag sequence into numeric indices. For the tokens, this can be done using the `tokenizer.convert_tokens_to_ids` method. For the tags, use the `role_to_id` dictionary. 
Each sequence must be a pytorch tensor of shape (1,128). You can convert a list of integer values like this `torch.tensor(token_ids, dtype=torch.long)`.

To keep everything organized, we will return a dictionary in the following format 

```
{'ids': token_tensor,
 'targets': tag_tensor,
 'mask': attention_mask_tensor,
 'pred': predicate_indicator_tensor}
```


(Hint: To debug these, read in the first annotation only / the first few annotations) 


In [11]:
from torch.utils.data import Dataset, DataLoader 
import numpy as np
class SrlData(Dataset):
    
    def __init__(self, filename):
        super(SrlData, self).__init__()
        self.max_len = 128 # the max number of tokens inputted to the transformer. 
        
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)        
        
        self.items = []
        with open (filename, 'r') as f:
            lines = f.readlines ()
            for line in range (0, len (lines), 4):
                
                sentence = lines [line + 1].strip()
                labels = lines [line + 3].strip()
                sentenceList, labelsList = tokenize_with_labels(sentence.split(), labels.split(), self.tokenizer)
                sentenceList = sentenceList [:126]
                labelsList = labelsList [:126]
                sentenceList.insert (0, '[CLS]')
                labelsList.insert (0, '[CLS]')
                sentenceList.append ('[SEP]')
                labelsList.append ('[SEP]')
        
                while (len (labelsList) < 128):
                    labelsList.append ('[PAD]')
                while (len (sentenceList) < 128):
                    sentenceList.append ('[PAD]')
                self.items.append ((sentenceList, labelsList))
                
                #self.items.append (tokenize_with_labels (sentenceList, labelsList, self.tokenizer))
                                                        
    def __len__(self):
        return len (self.items)
    
    def __getitem__(self, k):
        
        item = self.items [k]
        
        tokens = item [0]
        labels = item [1]
        
        attentionMask = []
        predicateMask = []
        
        for i in range (128):
            if (labels [i] == 'B-V'):
                predicateMask.append (1)
            else:
                predicateMask.append(0)
            if (labels [i] == '[PAD]'):
                attentionMask.append (0)
            else:
                attentionMask.append(1)
        
        attentionMask = np.array (attentionMask)
        predicateMask = np.array (predicateMask)
        tokensNum = tokenizer.convert_tokens_to_ids (tokens)
        labelsNum = np.zeros ((1, 128, 53))
        index = 0
        for label in labels:
            curr_arr = np.zeros (53)
            if (label in role_to_id and role_to_id [label] > 0):
                curr_arr [role_to_id [label]] = 1
            else:
                curr_arr [0] = 1
            #curr_arr [role_to_id [label] if role_to_id [label] > 0 else 0] = 1
            #labelsNum.append(role_to_id [label])
            labelsNum [0, index] = curr_arr
            index += 1
        '''labelsNum = np.zeros ((1, 128))
        index = 0
        for label in labels:
            if (label in role_to_id):
                labelsNum [0, index] = role_to_id [label]
            else:
                labelsNum [0, index] = -100
            index+= 1
        labelsNum = torch.tensor(labelsNum, dtype = torch.float).cuda()
        labelsNum = torch.unsqueeze (labelsNum, 0)
        print (labelsNum.shape)'''
            
        
        return {'ids': torch.reshape(torch.tensor (np.array(tokensNum), dtype = torch.long).cuda(), (1, 128)),
                'mask':  torch.reshape (torch.tensor (attentionMask, dtype = torch.long).cuda(), (1, 128)), 
                'targets': torch.tensor (np.array(labelsNum), dtype = torch.float).cuda(),
                'pred': torch.reshape(torch.tensor (predicateMask, dtype = torch.long).cuda(), (1, 128))
               }
        

In [12]:
# Reading the training data takes a while for the entire data because we preprocess all data offline
data = SrlData("propbank_train.tsv")

## 2. Model Definition

In [13]:
from torch.nn import Module, Linear, CrossEntropyLoss
from transformers import BertModel

We will define the pyTorch model as a subclass of the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class. The code for the model is provided for you. It may help to take a look at the documentation to remind you of how Module works. Take a look at how the huggingface BERT model simply becomes another sub-module. 

In [14]:
class SrlModel(Module):
    
    def __init__(self):
        
        super(SrlModel, self).__init__()
        
        self.encoder = BertModel.from_pretrained("bert-base-uncased")
    
        # The following two lines would freeze the BERT parameters and allow us to train the classifier by itself.
        # We are fine-tuning the model, so you can leave this commented out!
        # for param in self.encoder.parameters():
        #    param.requires_grad = False
        
        # The linear classifier head, see model figure in the introduction. 
        self.classifier = Linear(768, len(role_to_id))
                
        
    def forward(self, input_ids, attn_mask, pred_indicator):
    
        # This defines the flow of data through the model 
    
        # Note the use of the "token type ids" which represents the segment encoding explained in the introduction. 
        # In our segment encoding, 1 indicates the predicate, and 0 indicates everything else. 
        bert_output =  self.encoder(input_ids=input_ids, attention_mask=attn_mask, token_type_ids=pred_indicator)

        enc_tokens = bert_output[0] # the result of encoding the input with BERT
        logits = self.classifier(enc_tokens) #feed into the classification layer to produce scores for each tag.
        
        # Note that we are only interested in the argmax for each token, so we do not have to normalize 
        # to a probability distribution using softmax. The CrossEntropyLoss loss function takes this into account.
        # It essentially computes the softmax first and then computes the negative log-likelihood for the target classes. 
        return logits        

In [15]:
model = SrlModel().to('cuda') # create new model and store weights in GPU memory

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we are ready to try running the model with just a single input example to check if it is working correctly. Clearly it has not been trained, so the output is not what we expect. But we can see what the loss looks like for an initial sanity check. 

**TODO**: 
* Take a single data item from the dev set, as provided by your Dataset class defined above. Obtain the input token ids, attention mask, predicate indicator mask, and target labels. 
* Run the model on the ids, attention mask, and predicate mask like this: 

In [16]:
# pick an item from the dataset. Then run 
item = data.__getitem__ (43)
#outputs = model(item ['ids'], item ['mask'], item ['pred'])

**TODO**: 
Compute the loss on this one item only.
The initial loss should be close to -ln(1/num_labels)

Without training we would assume that all labels for each token (including the target label) are equally likely, so the negative log probability for the targets should be approximately $$-\ln(\frac{1}{\text{num_labels}}).$$ This is what the loss function should return on a single example. This is a good sanity check to run for any multi-class prediction problem. 

In [17]:
import math
-math.log(1 / len(role_to_id), math.e)

3.970291913552122

In [18]:
loss_function = CrossEntropyLoss(ignore_index = 0, reduction='mean')

# complete this. Note that you still have to provide a (batch_size, input_pos) 
# tensor for each parameter, where batch_size =1

outputs = model(item ['ids'], item ['mask'], item ['pred'])
#print (torch.argmax(item ['targets'][0], dim = 1))
#print (item ['ids'].shape)
#print (outputs.shape)
#print (item['targets'].shape)
loss = loss_function(outputs.transpose(2,1), torch.argmax(item['targets'], dim = 2))
print(loss.item())#this should be approximately the score from the previous cell


3.7455992698669434


**TODO**: At this point you should also obtain the actual predictions by taking the argmax over each position.
The result should look something like this (values will differ).

```
tensor([[ 1,  4,  4,  4,  4,  4,  5, 29, 29, 29,  4, 28,  6, 32, 32, 32, 32, 32,
         32, 32, 30, 30, 32, 30, 32,  4, 32, 32, 30,  4, 49,  4, 49, 32, 30,  4,
         32,  4, 32, 32,  4,  2,  4,  4, 32,  4, 32, 32, 32, 32, 30, 32, 32, 30,
         32,  4,  4, 49,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  6,  6, 32, 32,
         30, 32, 32, 32, 32, 32, 30, 30, 30, 32, 30, 49, 49, 32, 32, 30,  4,  4,
          4,  4, 29,  4,  4,  4,  4,  4,  4, 32,  4,  4,  4, 32,  4, 30,  4, 32,
         30,  4, 32,  4,  4,  4,  4,  4, 32,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4]], device='cuda:0')
```

Then use the id_to_role dictionary to decode to actual tokens. 

```
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'I-ARGM-TMP', 'O', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'O', '[SEP]', 'O', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'O', 'I-ARGM-TMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'O', 'O', 'O', 'I-ARG0', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'I-ARG2', 'O', 'I-ARG1', 'O', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
```

For now, just make sure you understand how to do this for a single example. Later, you will write a more formal function to do this once we have trained the model. 

In [22]:
torch.cuda.empty_cache()
predictions = torch.argmax (outputs, dim = 2)
actual_tokens = []
for pred in predictions [0]:
    pred_num =(int)(pred.cpu().detach().numpy())
    
    #print (pred.cpu().detach().numpy() + 2)
    actual_tokens.append (id_to_role[pred_num if pred_num != 0 else -100])
print (actual_tokens)

['[CLS]', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'B-V', 'I-V', 'O', '[SEP]', 'O', 'B-ARG1', 'O', 'O', 'O', 'B-ARG1', 'B-ARG1', 'O', 'B-ARG1', 'B-ARG1', 'B-ARG1', 'O', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-V', 'I-ARG1', 'I-ARG1', 'O', 'I-V', 'I-V', 'O', 'B-ARG2', 'B-V', 'O', 'O', 'O', 'B-V', 'B-V', 'O', 'O', 'O', 'O', 'I-V', 'B-ARG1', 'I-ARG1', 'B-V', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'B-ARG2', 'I-V', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-V', 'O', 'B-V', 'O', 'O', 'O', 'O', 'I-V', 'O', 'I-ARG1', 'I-ARG1', 'B-ARG1', 'O', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'I-ARG1', 'O', 'B-V', 'I-V', 'I-V', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-V', 'O', 'O', 'B-V', 'B-V', 'I-V', 'O', 'B-V', 'B-V', 'O', 'B-V', 'B-ARG1', 'O', 'B-ARG1', 'B-V', 'O', 'B-ARG2', 'I-V', 'B-V', 'B-V', 'B-V']


## 3. Training loop

pytorch provides a DataLoader class that can be wrapped around a Dataset to easily use the dataset for training. The DataLoader allows us to easily adjust the batch size and shuffle the data. 

In [15]:
from torch.utils.data import DataLoader
loader = DataLoader(data, batch_size = 32, shuffle = True)
#print (len(data))
torch.cuda.empty_cache()
#print (torch.cuda.memory_allocated(0))

The following cell contains the main training loop. The code should work as written and report the loss after each batch,
cumulative average loss after each 100 batches, and print out the final average loss after the epoch. 

**TODO**: Modify the training loop belowso that it also computes the accuracy for each batch and reports the 
average accuracy after the epoch. 
The accuracy is the number of correctly predicted token labels out of the number of total predictions. 
Make sure you exclude [PAD] tokens, i.e. tokens for which the target label is -100. It's okay to include [CLS] and [SEP] in the accuracy calculation. 

In [16]:
loss_function = CrossEntropyLoss(ignore_index = 0, reduction='mean')

LEARNING_RATE = 1e-05
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

device = 'cuda'
def train():
        
    """
    Train the model for one epoch.
    """
    tr_loss = 0 
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    #print ("hello")
    batch_num = 0
    total_correct1 = 0
    total_nonpad1 = 0
    
    for idx, batch in enumerate(loader):
        #torch.cuda.empty_cache()
        #Tracking training progress
        print (idx)
        # Get the encoded data for this batch and push it to the GPU
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        pred_mask = batch['pred'].to(device, dtype = torch.long)
        
        # Run the forward pass of the model
        logits = model(input_ids=ids[:, 0, :], attn_mask=mask[:, 0, :], pred_indicator=pred_mask[:, 0, :])
        #logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
        #loss = loss_function(logits.transpose(2,1), targets)
        loss = loss_function(logits.transpose(2,1), torch.argmax(targets[:, 0, :, :], dim = 2))
        tr_loss += loss.item()
        print("Batch loss: ", loss.item()) # can comment out if too verbose.
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            torch.cuda.empty_cache() # can help if you run into memory issues
            curr_avg_loss = tr_loss/nb_tr_steps
            print(f"Current average loss: {curr_avg_loss}")
        #print (torch.argmax(logits[0], dim = 1))
        #print ("-----------------")
        #print (torch.argmax(targets[0, 0], dim = 1))
        #print (len(torch.argmax(logits[0], dim = 1)))
        current_nonpad = 0
        current_correct = 0
        #print (logits.shape)
        for i in range (logits.shape[0]):
            for j in range (128):
                logit_val = torch.argmax(logits[i], dim = 1)[j].cpu().detach().numpy()
                target_val = torch.argmax(targets[i, 0], dim = 1)[j].cpu().detach().numpy()
                if (target_val != 0):
                    current_nonpad += 1
                    if (logit_val == target_val):
                        current_correct += 1
        #print ("yohoho")
        #print (current_correct)
        #print (current_nonpad)
        total_correct1 += current_correct
        total_nonpad1 += current_nonpad
        
        
        # Compute accuracy for this batch
        #print (torch.argmax(targets[:, 0, : , :], dim = 2).shape)
        #matching = torch.sum(torch.argmax(logits,dim=2) == targets[:, 0, : , :])
        #matching = torch.sum(torch.argmax(logits,dim=2) == torch.argmax(targets[:, 0, : , :], dim = 2)) 
        #predictions = torch.sum(torch.where(targets==-100,0,1))
        #total_correct += matching
        #print (torch.argmax(logits,dim=2) == torch.argmax(targets[:, 0, : , :], dim = 2))
        #predictions = torch.sum(torch.where(torch.argmax(targets[:, 0, : , :], dim = 2)==0,0,1))
        #print (matching)
        #print (predictions)
        # Run the backward pass to update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_num += 1
    
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accuracy = total_correct1/ total_nonpad1
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training epoch accuracy: {epoch_accuracy}")

Now let's train the model for one epoch. This will take a while (up to a few hours). 

In [17]:
train()

0
Batch loss:  3.923503875732422
Current average loss: 3.923503875732422
1
Batch loss:  3.876579761505127
2
Batch loss:  3.813473701477051
3
Batch loss:  3.6965818405151367
4
Batch loss:  3.561807632446289
5
Batch loss:  3.628596544265747
6
Batch loss:  3.4728047847747803
7
Batch loss:  3.385468006134033
8
Batch loss:  3.3643476963043213
9
Batch loss:  3.1958391666412354
10
Batch loss:  3.1160101890563965
11
Batch loss:  3.088334083557129
12
Batch loss:  3.095611810684204
13
Batch loss:  2.934330463409424
14
Batch loss:  2.848954439163208
15
Batch loss:  2.962498426437378
16
Batch loss:  2.6050608158111572
17
Batch loss:  2.5259156227111816
18
Batch loss:  2.248781442642212
19
Batch loss:  2.3711414337158203
20
Batch loss:  2.0513150691986084
21
Batch loss:  2.4130642414093018
22
Batch loss:  2.149528741836548
23
Batch loss:  2.1290132999420166
24
Batch loss:  1.9036186933517456
25
Batch loss:  2.0126991271972656
26
Batch loss:  1.9786958694458008
27
Batch loss:  1.9604583978652954
28


In my experiments, I found that two epochs are needed for good performance. 

In [41]:
train()

0
Batch loss:  0.19435900449752808
Current average loss: 0.19435900449752808
1
Batch loss:  0.10991370677947998
2
Batch loss:  0.1653437465429306
3
Batch loss:  0.25033044815063477
4
Batch loss:  0.24162934720516205
5
Batch loss:  0.16915221512317657
6
Batch loss:  0.1801658421754837
7
Batch loss:  0.10917738080024719
8
Batch loss:  0.1450423002243042
9
Batch loss:  0.3471071124076843
10
Batch loss:  0.23618170619010925
11
Batch loss:  0.271192729473114
12
Batch loss:  0.21026289463043213
13
Batch loss:  0.1452210694551468
14
Batch loss:  0.2163105607032776
15
Batch loss:  0.11051787436008453
16
Batch loss:  0.19311895966529846
17
Batch loss:  0.18312720954418182
18
Batch loss:  0.15448901057243347
19
Batch loss:  0.15633535385131836
20
Batch loss:  0.2712506949901581
21
Batch loss:  0.18048642575740814
22
Batch loss:  0.15217143297195435
23
Batch loss:  0.20046770572662354
24
Batch loss:  0.22993873059749603
25
Batch loss:  0.2466251105070114
26
Batch loss:  0.17636093497276306
27
Bat

I ended up with a training loss of about 0.19 and a training accuracy of 0.94. Specific values may differ. 

At this point, it's a good idea to save the model (or rather the parameter dictionary) so you can continue evaluating the model without having to retrain. 

In [42]:
torch.save(model.state_dict(), "srl_model_fulltrain_2epoch_finetune_1e-05.pt")

## 4. Decoding

In [19]:
# Optional step: If you stopped working after part 3, first load the trained model 

model = SrlModel().to('cuda') 
model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
model = model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**TODO (this is the fun part)**: Now that we have a trained model, let's try labeling an unseen example sentence. Complete the functions decode_output and label_sentence below. decode_output takes the logits returned by the model, extracts the argmax to obtain the label predictions for each token, and then translate the result into a list of string labels. 

label_sentence takes a list of input tokens and a predicate index, prepares the model input, call the model and then call decode_output to produce a final result. 

Note that you have already implemented all components necessary (preparing the input data from the token list and predicate index, decoding the model output). But now you are putting it together in one convenient function. 

In [43]:
tokens = "A U. N. team spent an hour inside the hospital , where it found evident signs of shelling and gunfire .".split()

In [44]:
def decode_output(logits): # it will be useful to have this in a separate function later on
    """
    Given the model output, return a list of string labels for each token.
    """
    output_list = []
    
    for i in range (1, 128):
        #print (torch.argmax (logits [0, i]))
        if (id_to_role[(int)(torch.argmax (logits [0, i]).cpu().detach().numpy())] == '[SEP]'):
            break
        output_list.append (id_to_role[(int)(torch.argmax (logits [0, i]).cpu().detach().numpy())])
    return output_list
    

In [45]:
def label_sentence(tokens, pred_idx):
    
    # complete this function to prepare token_ids, attention mask, predicate mask, then call the model. 
    # Decode the output to produce a list of labels.
    
    
    tokens_copy = tokens.copy()
    tokens_copy.insert(0, '[CLS]')
    tokens_copy.append ('[SEP]')
    while (len (tokens_copy) < 128):
        tokens_copy.append ('[PAD]')
    attention_mask = np.zeros ((1, 128))
    predicate_mask = np.zeros ((1, 128))
    for i in range (128):
        if (tokens_copy [i] == '[PAD]'):
            attention_mask [0, i] = 0
        else:
            attention_mask [0, i] = 1
    token_ids = tokenizer.convert_tokens_to_ids (tokens_copy)
    token_ids = torch.reshape(torch.tensor (np.array(token_ids), dtype = torch.long).cuda(), (1, 128))
    attention_mask = torch.reshape(torch.tensor (np.array(attention_mask), dtype = torch.long).cuda(), (1, 128))
    
    predicate_mask [0, np.min((pred_idx + 1, 127))] = 1
    predicate_mask = torch.reshape(torch.tensor (np.array(predicate_mask), dtype = torch.long).cuda(), (1, 128))
    
    logits = model (token_ids, attention_mask, predicate_mask)
    #print (predicate_mask)
    return decode_output (logits)
        

In [46]:
# Now you should be able to run

label_test = label_sentence(tokens, 13) # Predicate is "found"
list(zip(tokens, label_test))

[('A', 'O'),
 ('U.', 'O'),
 ('N.', 'O'),
 ('team', 'O'),
 ('spent', 'O'),
 ('an', 'O'),
 ('hour', 'O'),
 ('inside', 'O'),
 ('the', 'B-ARGM-LOC'),
 ('hospital', 'I-ARGM-LOC'),
 (',', 'O'),
 ('where', 'B-ARGM-LOC'),
 ('it', 'B-ARG0'),
 ('found', 'B-V'),
 ('evident', 'B-ARG1'),
 ('signs', 'I-ARG1'),
 ('of', 'I-ARG1'),
 ('shelling', 'I-ARG1'),
 ('and', 'I-ARG1'),
 ('gunfire', 'I-ARG1'),
 ('.', 'O')]

The expected output is somethign like this: 
```   
 ('A', 'O'),
 ('U.', 'O'),
 ('N.', 'O'),
 ('team', 'O'),
 ('spent', 'O'),
 ('an', 'O'),
 ('hour', 'O'),
 ('inside', 'O'),
 ('the', 'B-ARGM-LOC'),
 ('hospital', 'I-ARGM-LOC'),
 (',', 'O'),
 ('where', 'B-ARGM-LOC'),
 ('it', 'B-ARG0'),
 ('found', 'B-V'),
 ('evident', 'B-ARG1'),
 ('signs', 'I-ARG1'),
 ('of', 'I-ARG1'),
 ('shelling', 'I-ARG1'),
 ('and', 'I-ARG1'),
 ('gunfire', 'I-ARG1'),
 ('.', 'O'),
```


### 5. Evaluation 1: Token-Based Accuracy
We want to evaluate the model on the dev or test set. 

In [20]:
dev_data = SrlData("propbank_dev.tsv") # Takes a while because we preprocess all data offline

In [26]:
from torch.utils.data import DataLoader
loader = DataLoader(dev_data, batch_size = 1, shuffle = False)
device = 'cuda'

In [ ]:
# Optional: Load the model again if you stopped working prior to this step. 
# model = SrlModel()
# model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
# model = mode.to('cuda')

**TODO**: Complete the evaluate_token_accuracy function below. The function should iterate through the items in the data loader (see training loop in part 3). Run the model on each sentence/predicate pair and extract the predictions.

For each sentence, count the correct predictions and the total predictions. Finally, compute the accuracy as #correct_predictions / #total_predictions

Careful: You need to filter out the padded positions ([PAD] target tokens), as well as [CLS] and [SEP]. It's okay to include [B-V] in the count though. 

In [51]:
def evaluate_token_accuracy(model, loader):
    
    model.eval() # put model in evaluation mode
    
    # for the accuracy 
    total_correct = 0 # number of correct token label predictions. 
    total_predictions = 0 # number of total predictions = number of tokens in the data. 
    
    # iterate over the data here. 
    
    for idx, batch in enumerate(loader):
        #torch.cuda.empty_cache()
        # Get the encoded data for this batch and push it to the GPU
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        pred_mask = batch['pred'].to(device, dtype = torch.long)
        
        logits = model(input_ids=ids[:, 0, :], attn_mask=mask[:, 0, :], pred_indicator=pred_mask[:, 0, :])
        
        
        for i in range (logits.shape[0]):
            for j in range (128):
                logit_val = torch.argmax(logits[i], dim = 1)[j].cpu().detach().numpy()
                target_val = torch.argmax(targets[i, 0], dim = 1)[j].cpu().detach().numpy()
                if (target_val != 0 and target_val != role_to_id ['[CLS]'] and target_val != role_to_id ['[SEP]']):
                    total_predictions += 1
                    if (logit_val == target_val):
                          total_correct += 1
    
    
    acc = total_correct / total_predictions
    print(f"Accuracy: {acc}")
evaluate_token_accuracy (model, loader)

Accuracy: 0.932360471620982


### 6. Span-Based evaluation 

While the accuracy score in part 5 is encouraging, an accuracy-based evaluation is problematic for two reasons. First, most of the target labels are actually O. Second, it only tells us that per-token prediction works, but does not directly evaluate the SRL performance. 

Instead, SRL systems are typically evaluated on micro-averaged precision, recall, and F1-score for predicting labeled spans. 

More specifically, for each sentence/predicate input, we run the model, decode the output, and extract a set of labeled spans (from the output and the target labels). These spans are (i,j,label) tuples.  

We then compute the true_positives, false_positives, and false_negatives based on these spans. 

In the end, we can compute 

* Precision:  true_positive / (true_positives + false_positives)  , that is the number of correct spans out of all predicted spans. 

* Recall: true_positives / (true_positives + false_negatives) , that is the number of correct spans out of all target spans. 

* F1-score:   (2 * precision * recall) / (precision + recall)


For example, consider 

| |[CLS]|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|--||---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
||0|1|2|3|4|5|6|7|8|9|1O|11|12|13|14|15|
|target|[CLS]|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|prediction|[CLS]|B-ARG1|I-ARG1|B-V|I-ARG2|I-ARG2|O|O|O|O|O|O|O|O|B-ARGM-TMP|O|

The target spans are (1,2,"ARG1"), and (4,8,"ARG2").

The predicted spans would be (1,2,"ARG1"), (14,14,"ARGM-TMP"). Note that in the prediction, there is no proper ARG2 span because we are missing the B-ARG2 token, so this span should not be created. 

So for this sentence we woudl get: true_positives: 1 false_positives: 1 false_negatives: 1

*TODO*: Complete the function evaluate_spans that performs the span-based evaluation on the given model and data loader. You can use the provided extract_spans function, which returns the spans as a dictionary. For example
{(1,2): "ARG1", (4,8):"ARG2"} 

In [34]:
def extract_spans(labels):
    spans = {} # map (start,end) ids to label
    current_span_start = 0
    current_span_type = ""
    inside = False
    for i, label in enumerate(labels):
        if label.startswith("B"):            
            if inside: 
                if current_span_type != "V":
                    spans[(current_span_start,i)] = current_span_type            
            current_span_start = i
            current_span_type = label[2:]
            inside = True
        elif inside and label.startswith("O"):
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
        elif inside and label.startswith("I") and label[2:] != current_span_type:            
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
    return spans
                        

In [41]:
def evaluate_spans(model, loader):
    
    
    total_tp = 0
    total_fp = 0
    total_fn = 0
        
    for idx, batch in enumerate(loader):
        torch.cuda.empty_cache()
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        pred_mask = batch['pred'].to(device, dtype = torch.long)
        
        logits = model(input_ids=ids[:, 0, :], attn_mask=mask[:, 0, :], pred_indicator=pred_mask[:, 0, :])
        logits_list = []
        targets_list = []

        found_target_sep = False
        found_logit_sep = False
        
        for i in range (logits.shape[0]):
            for j in range (1, 128):
                logit_val = torch.argmax(logits[i], dim = 1)[j].cpu().detach().numpy()
                target_val = torch.argmax(targets[i, 0], dim = 1)[j].cpu().detach().numpy()
                if (target_val == 0):
                    target_val = -100
                if (logit_val == 0):
                    logit_val = -100
                    
                if (id_to_role [(int)(logit_val)] == '[SEP]'):
                    found_logit_sep = True
                if (not found_logit_sep):
                    logits_list.append (id_to_role [(int)(logit_val)])
                    
                if (id_to_role [(int)(target_val)] == '[SEP]'):
                    found_target_sep = True
                if (not found_target_sep):
                    targets_list.append (id_to_role [(int)(target_val)])
                    
                if (found_logit_sep and found_target_sep):
                    break
        
        logit_spans = extract_spans (logits_list)
        target_spans = extract_spans (targets_list)
        
        for target in target_spans:
            if (target not in logit_spans):
                total_fn += 1
            else:
                if (target_spans [target] == logit_spans [target]):
                    total_tp += 1
                else:
                    total_fn += 1
                    
        for logit in logit_spans:
            if (logit not in target_spans):
                total_fp += 1
    
    total_p = total_tp / (total_tp + total_fp)
    total_r = total_tp / (total_tp + total_fn)
    total_f = (2 * total_p *total_r) / (total_p + total_r)
            
    print(f"Overall P: {total_p}  Overall R: {total_r}  Overall F1: {total_f}")
    
evaluate_spans(model, loader)    

Overall P: 0.8601658831352146  Overall R: 0.8296129612961296  Overall F1: 0.8446132082495719


In my evaluation, I got an F score of 0.82  (which slightly below the state-of-the art in 2018)

### OPTIONAL: 

Repeat the span-based evaluation, but print out precision/recall/f1-score for each role separately.